In [19]:
import numpy as np
import pandas as pd
import scipy
import string
import csv

In [80]:
df = pd.read_csv(filepath_or_buffer = 'best_features.csv', index_col = False) 

In [86]:
df

,D3,D0,D4,D8,F3,D6,F4,F7,F5,F6,N5,N3,N4,N7,N8,F9
0,ln_O3_int_lead_5,ln_temp_int_lag_90,Temp_int_lag_90,ln_O3_int_lead_5,ln_o3_rh_int_lead_5,Temp_int_lag_90,temp_cu,days from start,e2v03,ln_o3_rh_int_lead_5,Temp,ln_O3_int_lead_5,ln_O3_int_lead_5,Temp,ln_o3_rh_int_lead_5,ln_o3_rh_int_lead_5
1,Temp_int_lag_90,ln_O3_int_lead_5,ln_O3_int_lead_5,temp_cu,temp_rh_int_lead_30,ln_O3_int_lead_5,ln_O3,ln_O3,days from start,temp_rh_int_lead_90,ln_O3,temp_cu,temp_cu,Rh,temp_rh_int_lead_30,03_mult_rh
2,ln_o3_rh_int_lag_5,03_sq_mult_rh_sq,ln_rh_int_lag_90,o3_temp_rh_int_lead_90,o3_temp_slope_lag_90,ln_O3_int_lead_90,ln_03_mult_ln_rh_&_ln_temp,temp_cu,temp_cu,o3_rh_int_lag_5,temp_sq,temp_rh_int_lead_90,o3_temp_rh_int_lead_5,days from start,o3_temp_slope_lag_90,ln_temp_rh_int_lead_90
3,temp_sq,Temp_int_lag_90,e2v03_int_lead_5,ln_O3_int_lag_5,o3_rh_int_lag_5,ln_O3,ln_o3_temp_rh_int_lead_90,rh_cu,rh_cu,o3_temp_int_lag_90,03_sq_mult_temp_sq,e2v03_int_lead_5,e2v03_slope_lag_90,O3_cu,ln_o3_temp_rh_slope_lag_90,03_sq_mult_rh_sq
4,e2v03_slope_lag_30,03_sq_mult_temp_sq,temp_rh_int_lag_90,ln_rh_slope_lag_75,ln_O3_slope_lead_90,Temp_slope_lead_90,ln_o3_temp_int_lead_5,03_sq_mult_temp_sq,03_sq_mult_temp_sq,temp_cu,ln_O3_int_lag_30,ln_O3_int_lag_30,ln_o3_temp_int_lead_60,ln_O3,ln_temp_rh_slope_lag_90,ln_O3
5,ln_temp_rh_slope_lead_30,03_mult_temp,temp_rh_slope_lead_30,o3_temp_slope_lag_90,temp_rh_int_lag_5,ln_rh_slope_lead_15,Rh_int_lead_90,03_sq_mult_rh_sq,temp_mult_rh,ln_O3_int_lead_5,ln_O3_int_lead_30,e2v03_int_lag_30,Temp_slope_lag_30,temp_cu,temp_rh_int_lag_5,temp_sqrt
6,ln_rh_slope_lag_60,Temp_slope_lead_90,e2v03_int_lead_90,Rh_slope_lead_30,Temp_slope_lead_75,temp_cu,e2v03_slope_lead_60,ln_O3_slope_lead_45,03_sq_mult_rh_sq_&_temp_sq,Rh_int_lead_15,ln_O3_slope_lead_60,O3_cu,Rh_slope_lead_75,03_sq_mult_temp_sq,o3_rh_int_lead_90,ln_03_mult_ln_temp
7,Rh_slope_lag_60,ln_temp_slope_lead_90,Temp_slope_lag_15,o3_rh_int_lead_60,ln_temp_slope_lead_75,03_sq_mult_rh_sq,Rh_slope_lead_5,ln_O3_slope_lead_60,e2v03_int_lag_15,e2v03_int_lead_5,ln_O3_slope_lead_75,Rh_int_lead_90,Rh_slope_lead_90,03_sq_mult_rh_sq,ln_rh_slope_lead_90,O3_sqrt
8,Rh_slope_lag_15,Rh_slope_lead_60,ln_temp_rh_int_lead_90,o3_rh_int_lead_5,temp_sq_mult_rh_sq,days from start,e2v03_slope_lead_30,ln_O3_int_lead_5,e2v03_slope_lag_90,03_sq_mult_rh_sq_&_temp_sq,ln_O3_int_lead_5,o3_temp_int_lead_5,rh_cu,temp_sq_mult_rh_sq,Rh_slope_lead_45,ln_o3_temp_int_lead_5
9,ln_rh_slope_lead_90,e2v03_int_lead_5,o3_temp_int_lag_90,03_sq_mult_rh_sq_&_temp_sq,ln_o3_temp_rh_int_lag_30,o3_temp_slope_lag_90,e2v03_slope_lag_90,ln_O3_slope_lag_5,ln_O3_slope_lag_90,O3_sqrt,Temp_int_lead_15,Rh_slope_lag_15,Temp_int_lead_90,e2v03_int_lead_90,ln_o3_rh_int_lag_5,e2v03


####Get the top 5 features for each pod.

In [81]:
def count_top_features(num_feat):
    top_feat = []
    for a in df:
        top_feat.append(list(df[a][0:num_feat]))

    #flatten the list
    top_feat = [ b for a in top_feat for b in a ]

    #convert the list to a set to get unique values
    top_feat_set = set(top_feat)
    #convert the set back into a list
    top_feat_unique = list(top_feat_set)
    
    #count the number of times each feature appears in the top 5
    num_times = []
    feature = []
    for a in top_feat_unique:
        num_times.append(top_feat.count(a))
        feature.append(a)
    
    d = {'appearances': num_times, 'feature': feature }
    df_top_feat = pd.DataFrame(data=d)
    df_top_feat_sorted = df_top_feat.sort('appearances', ascending = False)
    return df_top_feat_sorted

In [82]:
df_top_five = count_top_features(5)
df_top_ten = count_top_features(10)
df_all = count_top_features(len(df))

In [120]:
df_top_feat = df_all[df_all.appearances >= 2]
best_15_feat = df_top_feat.feature[1:22]
print df_top_feat

     appearances                     feature
0            127                         NaN
89            12            ln_O3_int_lead_5
116            9                     temp_cu
40             6          03_sq_mult_temp_sq
34             6                       ln_O3
122            5             days from start
27             5            03_sq_mult_rh_sq
125            5             Temp_int_lag_90
16             5         ln_o3_rh_int_lead_5
75             4            e2v03_int_lead_5
58             4                       rh_cu
1              4        o3_temp_slope_lag_90
121            3          e2v03_slope_lag_45
78             3          o3_temp_int_lag_90
54             3                  03_mult_rh
49             3  03_sq_mult_rh_sq_&_temp_sq
43             3   ln_o3_temp_rh_int_lead_90
38             3                     temp_sq
69             3          e2v03_slope_lag_90
138            3          Temp_slope_lead_90
7              3         ln_rh_slope_lead_15
10        

###Find out which features never appear together.

In [117]:
dict1 = {}
for column in df.columns:
    for row in df[column]:
        if row not in dict1:
            dict1[row] = set()
        for row2 in df[column]:
            if row2 != row:
                dict1[row] |= {row2}

#get a list of all of the features that appear in the best features lists
all_features = dict1.keys()

dict2 = {}
for feature in df_top_feat['feature']:
    dict2[feature] = set()
    for feature2 in df_top_feat['feature']:
        if feature2 != feature and feature2 not in dict1[feature]:
            dict2[feature] |= {feature2}

In [118]:
conflict_list = dict2['ln_o3_rh_int_lead_5']

In [119]:
for best_feature in best_15_feat:
    print "'", best_feature, "',"
    conflict_list = dict2[best_feature]
    for feature in conflict_list:
        if feature in best_15_feat:
            print feature

' ln_O3_int_lead_5 ',
' temp_cu ',
' 03_sq_mult_temp_sq ',
' ln_O3 ',
' days from start ',
' 03_sq_mult_rh_sq ',
' Temp_int_lag_90 ',
' ln_o3_rh_int_lead_5 ',
' e2v03_int_lead_5 ',
' rh_cu ',
' o3_temp_slope_lag_90 ',
' e2v03_slope_lag_45 ',
' o3_temp_int_lag_90 ',
' 03_mult_rh ',
' 03_sq_mult_rh_sq_&_temp_sq ',
' ln_o3_temp_rh_int_lead_90 ',
' temp_sq ',
' e2v03_slope_lag_90 ',
' Temp_slope_lead_90 ',
' ln_rh_slope_lead_15 ',
' ln_o3_temp_int_lead_5 ',
